In [1]:
import livef1
from livef1.utils.constants import interpolation_map

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

2025-01-17 22:30:18,671 : livef1 : INFO :: Selected meeting/session is:
Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
Meeting Name : Belgian Grand Prix
Meeting Circuit Shortname : Spa-Francorchamps
2025-01-17 22:30:18,671 : livef1 : INFO :: The meeting was received successfully.
2025-01-17 22:30:18,674 : livef1 : INFO :: Selected meeting/session is:
Session Name : Race
2025-01-17 22:30:18,674 : livef1 : INFO :: The session was received successfully.


In [2]:
from livef1.adapters import livetimingF1_request, livetimingF1_getdata
from urllib.parse import urljoin
session.get_topic_names()

data = livetimingF1_getdata(
    urljoin(session.full_path, session.topic_names_info["CarData.z"]["StreamPath"]),
    stream=True
)

In [3]:
res = session.get_data(dataName="Position")
df_pos = pd.DataFrame(res.value)
df_pos["Utc"] = pd.to_datetime(df_pos["Utc"])
df_pos["timestamp"] = pd.to_datetime(df_pos["timestamp"])
df_pos["min"] = df_pos["Utc"].dt.minute
df_pos["hour"] = df_pos["Utc"].dt.hour
df_pos["sec"] = df_pos["Utc"].dt.second

2025-01-17 22:30:21,842 : livef1 : INFO :: Getting requested data.
Selected session : 2024 Belgian Grand Prix Race
Topic : Position.z
2025-01-17 22:30:22,254 : livef1 : INFO :: Data is successfully received.
2025-01-17 22:30:22,725 : livef1 : INFO :: Data is successfully parsed.
/var/folders/pd/1fxx8q0s4jb_8ry0vd5fdkg00000gn/T/ipykernel_24134/3877280185.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pos["timestamp"] = pd.to_datetime(df_pos["timestamp"])


In [65]:
res = session.get_data(dataName="Car_Data")
df_car = pd.DataFrame(res.value)
df_car["Utc"] = pd.to_datetime(df_car["Utc"])
df_car["timestamp"] = pd.to_timedelta(df_car["timestamp"])
df_car["min"] = df_car["Utc"].dt.minute
df_car["hour"] = df_car["Utc"].dt.hour
df_car["sec"] = df_car["Utc"].dt.second

2025-01-17 23:10:42,355 : livef1 : INFO :: Getting requested data.
Selected session : 2024 Belgian Grand Prix Race
Topic : CarData.z
2025-01-17 23:10:42,855 : livef1 : INFO :: Data is successfully received.
2025-01-17 23:10:43,759 : livef1 : INFO :: Data is successfully parsed.


In [67]:
df_car

,SessionKey,timestamp,Utc,DriverNo,rpm,speed,n_gear,throttle,brake,drs,min,hour,sec
0,9574,0 days 00:02:28.746000,2024-07-28 12:11:06.723344100+00:00,1,0,0,0,0,0,0,11,12,6
1,9574,0 days 00:02:28.746000,2024-07-28 12:11:06.723344100+00:00,2,0,0,0,0,0,0,11,12,6
2,9574,0 days 00:02:28.746000,2024-07-28 12:11:06.723344100+00:00,3,0,0,0,0,0,0,11,12,6
3,9574,0 days 00:02:28.746000,2024-07-28 12:11:06.723344100+00:00,4,0,0,0,0,0,0,11,12,6
4,9574,0 days 00:02:28.746000,2024-07-28 12:11:06.723344100+00:00,10,0,0,0,0,0,0,11,12,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
607115,9574,0 days 02:17:07.573000,2024-07-28 14:25:45.550120800+00:00,44,0,0,0,104,104,0,25,14,45
607116,9574,0 days 02:17:07.573000,2024-07-28 14:25:45.550120800+00:00,55,0,0,0,0,0,0,25,14,45
607117,9574,0 days 02:17:07.573000,2024-07-28 14:25:45.550120800+00:00,63,0,0,0,104,104,0,25,14,45
607118,9574,0 days 02:17:07.573000,2024-07-28 14:25:45.550120800+00:00,77,3148,16,1,0,0,0,25,14,45


In [68]:
# display(df_pos[df_pos['DriverNo'] == "44"].iloc[168:1000].head(10))
# display(df_car[df_car['DriverNo'] == "44"].head(10))

# df_44 = df_pos[df_pos['DriverNo'] == "44"].reset_index()
# df_44["time_diff"] = (df_44["Utc"] - df_44["Utc"].shift(1)).dt.total_seconds()
# df_44[df_44["time_diff"] < 0.1]
# df_44.loc[108:118]
df = df_car.set_index(["Utc","DriverNo"]).join(df_pos.set_index(["Utc","DriverNo"]), rsuffix="_pos", how="outer").reset_index()

In [85]:
df[~df["SessionKey"].isnull()]
df

,Utc,DriverNo,SessionKey,timestamp,rpm,speed,n_gear,throttle,brake,drs,...,sec,SessionKey_pos,timestamp_pos,Status,X,Y,Z,min_pos,hour_pos,sec_pos
0,2024-07-28 12:10:22.787731300+00:00,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
1,2024-07-28 12:10:22.787731300+00:00,10,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
2,2024-07-28 12:10:22.787731300+00:00,11,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
3,2024-07-28 12:10:22.787731300+00:00,14,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
4,2024-07-28 12:10:22.787731300+00:00,16,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232135,2024-07-28 14:25:45.550120800+00:00,44,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,...,45.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232136,2024-07-28 14:25:45.550120800+00:00,55,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232137,2024-07-28 14:25:45.550120800+00:00,63,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,...,45.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232138,2024-07-28 14:25:45.550120800+00:00,77,9574.0,0 days 02:17:07.573000,3148.0,16.0,1.0,0.0,0.0,0.0,...,45.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
import fastf1
session = fastf1.get_session(2024, "Belgium", 'R')
session.load()
tel44 = session.laps.pick_driver("44").get_telemetry()

req         WARNING 	DEFAULT CACHE ENABLED! (87.37 MB) /Users/goktugocal/Library/Caches/fastf1
core           INFO 	Loading data for Belgian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '81', '16', '1', '4', '55', '11', '14', '31', '3', '18', '23', '10', '20', '7

In [98]:
merged_df = tel44[["SessionTime","X","Y","Z"]].set_index("SessionTime").join(df_44[["timestamp","X","Y","Z"]].set_index("timestamp"), rsuffix="_LF1", how="outer")

In [99]:
merged_df[~merged_df["X"].isnull() & ~merged_df["X_LF1"].isnull()]
tel44

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
2,2024-07-28 13:03:52.742,0 days 00:55:14.764000,,0.264167,0 days 00:00:00,8090,3,1,20,False,1,interpolation,0.123562,4.049413e-07,OnTrack,-940,2070,4153
3,2024-07-28 13:03:52.748,0 days 00:55:14.770000,,0.264167,0 days 00:00:00.006000,7383,6,1,21,False,1,pos,0.139648,4.576584e-07,OnTrack,-941,2071,4154
4,2024-07-28 13:03:53.588,0 days 00:55:15.610000,,0.264167,0 days 00:00:00.846000,5968,12,1,23,False,1,pos,4.451260,1.458780e-05,OnTrack,-948,2081,4154
5,2024-07-28 13:03:53.693,0 days 00:55:15.715000,,0.264167,0 days 00:00:00.951000,4553,19,1,25,False,1,car,5.277778,1.729649e-05,OnTrack,-950,2085,4153
6,2024-07-28 13:03:53.808,0 days 00:55:15.830000,,0.264167,0 days 00:00:01.066000,4736,24,1,28,False,1,pos,6.256335,2.050344e-05,OnTrack,-955,2092,4154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36381,2024-07-28 14:23:50.050,0 days 02:15:12.072000,,3.505556,0 days 01:19:57.308000,10085,214,5,84,False,8,pos,305098.550304,9.998779e-01,OnTrack,-334,1154,4133
36382,2024-07-28 14:23:50.149,0 days 02:15:12.171000,,3.505556,0 days 01:19:57.407000,10230,218,6,69,False,8,car,305104.586389,9.998976e-01,OnTrack,-361,1196,4133
36383,2024-07-28 14:23:50.309,0 days 02:15:12.331000,,3.505556,0 days 01:19:57.567000,10144,219,6,53,False,8,car,305114.319722,9.999295e-01,OnTrack,-404,1260,4135
36384,2024-07-28 14:23:50.510,0 days 02:15:12.532000,,3.505556,0 days 01:19:57.768000,10063,217,6,33,False,8,pos,305126.397329,9.999691e-01,OnTrack,-456,1339,4138


In [87]:
df_44 = df[(df["DriverNo"] == "44")]
df_44

,Utc,DriverNo,SessionKey,timestamp,rpm,speed,n_gear,throttle,brake,drs,...,sec,SessionKey_pos,timestamp_pos,Status,X,Y,Z,min_pos,hour_pos,sec_pos
15,2024-07-28 12:10:22.787731300+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,22.0
35,2024-07-28 12:10:23.107682700+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,23.0
55,2024-07-28 12:10:23.307652300+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,23.0
75,2024-07-28 12:10:23.547615800+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:45.570,OnTrack,0.0,0.0,0.0,10.0,12.0,23.0
95,2024-07-28 12:10:23.927558+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 00:01:46.630,OnTrack,0.0,0.0,0.0,10.0,12.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232055,2024-07-28 14:25:44.870164400+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 02:17:06.893,OnTrack,1050.0,2542.0,4143.0,25.0,14.0,44.0
1232075,2024-07-28 14:25:44.910215600+00:00,44,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,...,44.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232095,2024-07-28 14:25:45.230168200+00:00,44,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,...,45.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232115,2024-07-28 14:25:45.330094500+00:00,44,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9574.0,2025-01-17 02:17:07.353,OnTrack,1050.0,2542.0,4143.0,25.0,14.0,45.0


In [32]:
df_44["X"] = df_44.reset_index()["X"].astype(float).interpolate(method = "quadratic", order = 2).values
df_44["Y"] = df_44.reset_index()["Y"].astype(float).interpolate(method = "quadratic", order = 2).values

/var/folders/pd/1fxx8q0s4jb_8ry0vd5fdkg00000gn/T/ipykernel_24134/4038674442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_44["X"] = df_44.reset_index()["X"].astype(float).interpolate(method = "quadratic", order = 2).values
/var/folders/pd/1fxx8q0s4jb_8ry0vd5fdkg00000gn/T/ipykernel_24134/4038674442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_44["Y"] = df_44.reset_index()["Y"].astype(float).interpolate(method = "quadratic", order = 2).values


In [43]:
df_44.dropna(subset="timestamp")

(df_44.Utc - df_44.Utc.shift(1)).mean()

Timedelta('0 days 00:00:00.131850183')

In [55]:
df_temp.dropna(subset="speed")

,Utc,DriverNo,SessionKey,timestamp,rpm,speed,n_gear,throttle,brake,drs,...,sec,SessionKey_pos,timestamp_pos,Status,X,Y,Z,min_pos,hour_pos,sec_pos
440015,2024-07-28 12:58:46.811878100+00:00,44,9574.0,0 days 00:50:09.394000,3981.0,0.0,0.0,0.0,0.0,1.0,...,46.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440055,2024-07-28 12:58:47.211819300+00:00,44,9574.0,0 days 00:50:09.394000,4005.0,0.0,0.0,0.0,0.0,1.0,...,47.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440095,2024-07-28 12:58:47.371795800+00:00,44,9574.0,0 days 00:50:09.394000,3997.0,0.0,0.0,0.0,0.0,1.0,...,47.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440115,2024-07-28 12:58:47.611760300+00:00,44,9574.0,0 days 00:50:10.475000,3981.0,0.0,0.0,0.0,0.0,1.0,...,47.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440155,2024-07-28 12:58:47.811730100+00:00,44,9574.0,0 days 00:50:10.475000,4065.0,0.0,0.0,0.0,0.0,1.0,...,47.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469815,2024-07-28 13:02:00.653062700+00:00,44,9574.0,0 days 00:53:23.875000,10319.0,251.0,7.0,67.0,0.0,1.0,...,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469855,2024-07-28 13:02:00.853032700+00:00,44,9574.0,0 days 00:53:23.875000,10293.0,251.0,7.0,67.0,0.0,1.0,...,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469895,2024-07-28 13:02:01.132991500+00:00,44,9574.0,0 days 00:53:23.875000,10376.0,252.0,7.0,68.0,0.0,1.0,...,1.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469915,2024-07-28 13:02:01.332962100+00:00,44,9574.0,0 days 00:53:23.875000,10380.0,253.0,7.0,68.0,0.0,1.0,...,1.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_temp["speed"]

440015      0.0
440035      NaN
440055      0.0
440075      NaN
440095      0.0
          ...  
469915    253.0
469935      NaN
469955    253.0
469975      NaN
469995      NaN
Name: speed, Length: 1500, dtype: float64

In [63]:
from plotly import express as px
df_temp = df[(df["DriverNo"] == "44")].iloc[22000:23500]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [35]:
df_temp = df_44.iloc[22000:23900][~df_44["SessionKey"].isnull()]
px.scatter(x = df_temp.X, y = df_temp.Y)

/var/folders/pd/1fxx8q0s4jb_8ry0vd5fdkg00000gn/T/ipykernel_24134/1169601629.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [36]:
df_temp = df_44.iloc[22000:23900]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [39]:
df_44["speed"] = df_44.reset_index()["speed"].astype(float).interpolate(method = "polynomial", order = 2).values
df_temp = df_44.iloc[22000:23500]
df_temp["fill"] = df_temp["DriverNo_pos"].isnull()
px.scatter(df_temp, x = df_temp.index, y = "speed", color="fill")

/var/folders/pd/1fxx8q0s4jb_8ry0vd5fdkg00000gn/T/ipykernel_24134/3027563336.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError: 'DriverNo_pos'

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_weather = pd.DataFrame(res.value)
df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"])

In [7]:
idx_cols = ["hour","min","sec","DriverNo"]
df = df_car.set_index(idx_cols).join(df_pos.set_index(idx_cols), rsuffix="_")

In [ ]:
c = pd.to_datetime(df_car[(df_car["min"] == 2) & (df_car["hour"] == 0)].timestamp).unique()
p = pd.to_datetime(df_w[(df_w["min"] == 2) & (df_w["hour"] == 0) & (df_w["sec"] > 27)].timestamp).unique()

for i in c-p:
    print(i.total_seconds())

In [8]:
df_all = df.reset_index().set_index("timestamp").join(df_weather.set_index("timestamp"), rsuffix="_w_", how="outer")

In [ ]:
df_car[df_car["DriverNo"] == "44"]

In [ ]:
df.reset_index()[df.reset_index()["DriverNo"] == "44"]

In [ ]:
df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

## Weather Data

In [9]:
for col in interpolation_map:
    df_all[col] = df_all.reset_index()[col].astype(float).interpolate(method = interpolation_map[col], order = 2).values

In [ ]:
import matplotlib.pyplot as plt

df_temp = df_all[~df_all["DriverNo"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
df_temp = df_all[df_all["DriverNo"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
plt.show()

In [ ]:
df_all.dropna(subset="DriverNo")

## Position Data

In [ ]:
df_all.dtypes

In [ ]:
df_temp = df_all[~df_all["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
df_temp = df_all[df_all["X"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
plt.show()

In [ ]:
df_44 = df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["speed"])

df_temp = df_44[df_44["X"].isnull()]
plt.scatter(df_temp.index, df_temp["speed"])
plt.show()


df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.timestamp), df_temp["speed"])
plt.show()

In [ ]:
pd.to_datetime(df_44.timestamp).dt.time - pd.to_datetime(df_44.timestamp).shift(1).dt.time

In [ ]:
df_44.head(50)

In [ ]:
(pd.to_datetime(df_44.timestamp) - pd.to_datetime(df_44.timestamp).shift(1)).dt.total_microseconds()

In [ ]:
df_44

In [ ]:
for col in df_weather.columns[2:]:
    plt.figure(figsize=(20,5))
    plt.scatter(pd.to_datetime(df_all.index).values, df_all[col].astype(float), label = col)
    plt.legend()
    plt.show()

In [ ]:
na_list = [np.nan] * 100
test_ts = pd.Series([-1] + na_list + [-1] + na_list + [-1] + na_list + [0] + na_list + [1]+ na_list + [1] + na_list + [1])

plt.figure(figsize=(20,5))
plt.scatter(test_ts.index, test_ts, label = "plain")
plt.plot(test_ts.index, test_ts.interpolate(method = "linear"), label = "linear")
plt.show()

for method in ['nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

for method in ['krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

# plt.figure(figsize=(20,5))
# for d in [2,3,5]:
#     plt.plot(test_ts.index, test_ts.interpolate(method = "spline", order = d), label = "spline_" + str(d))
# plt.legend()
# plt.show()



In [ ]:
df_temp = df_all.copy()
df_temp.index = pd.to_datetime(df_all.index).values

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "linear"), label = "linear")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "polynomial", order = 2), label = "polynomial")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "spline", order = 2), label = "spline")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float), label = "plain")
plt.show()



In [ ]:
len(df_temp)
df_viz

In [ ]:
for d in [2,3,5,7,9,15,25,35]:
    try:
        df_viz = df_temp["WindSpeed"].astype(float).interpolate(method = "linear", order = d, limit_direction="both").reset_index().iloc[500000:1000000]
        print(d)
        plt.figure(figsize=(20,5))
        plt.scatter(df_viz["index"], df_viz["WindSpeed"], label = str(d))
        plt.show()
    except:
        pass

In [ ]:
df_all.reset_index()[~df_all.reset_index()["AirTemp"].isnull()]

In [ ]:
df_all.reset_index().iloc[2373112-25:2373112+25]

In [14]:
import numpy as np

cols = df_weather.columns[2:]
df_all.loc[:,cols] = df_all.loc[:,cols].ffill()


In [ ]:
df_all.dropna(subset="Channels").dtypes

In [ ]:
df_all

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_w = pd.DataFrame(res.value)

In [ ]:
res = session.get_data(dataName="Track_Status")
df = pd.DataFrame(res.value)
# df["Utc"] = pd.to_datetime(df["Utc"])
# df["duration"] = df["Utc"] - df["Utc"].shift(1)
df

In [ ]:
res = session.get_data(dataName="Tyre_Stints")
df = pd.DataFrame(res.value)
df

In [ ]:
df

In [ ]:
res = session.get_data(dataName="Weather_Data")
df = pd.DataFrame(res.value)
df

In [ ]:
from livef1.adapters.livetimingf1_adapter import livetimingF1_getdata

data = livetimingF1_getdata("https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/PitLaneTimeCollection.jsonStream", stream=True)
data

for key, value in data.items():
    if "_deleted" in value["PitTimes"].keys():
        for deleted_driver in value["PitTimes"]["_deleted"]:
            record = {
                "session_key": 0,
                "timestamp": key,
                "_deleted": deleted_driver
            }
            print(record)
            
    else:
        for driver_no, info in value["PitTimes"].items():
            print(info)
            record = {
                "session_key": 0,
                "timestamp": key,
                **info
            }
            print(record)
    
    print("----------")

In [ ]:
res = session.get_data(dataName="Pit_Lane_Times")
df = pd.DataFrame(res.value)
df

In [ ]:
import fastf1

session = fastf1.get_session(2024, 'Belgian', 'Race')
session.load()

In [ ]:
help(session)

In [ ]:
session.weather_data

In [ ]:
session.laps

In [ ]:
session.laps[~session.laps["PitInTime"].isnull()].sort_values("Time")

In [ ]:
session.laps[~session.laps["PitOutTime"].isnull()].sort_values("Time")

In [ ]:
session.race_control_messages

In [ ]:
session.results

In [ ]:
session.car_data["1"]